In [1]:
%run models.ipynb
%run datasets.ipynb

In [13]:
import dalex as dx
import numpy as np
import shap

/home/edsuch21/anaconda3/envs/myDalex/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
svm = create_model('svm', kernel='linear')
xgboost = create_model('xgboost')

## Szacowanie ceny mieszkań

In [4]:
X_train, X_test, y_train, y_test = get_test_train('house_price')

In [5]:
model_predicting_method = lambda m, d: m.predict(d)

In [6]:
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.4950936198067202

In [7]:
xgboost.fit(X_train, y_train)
xgboost.score(X_test, y_test)

0.6829726555492395

In [8]:
explainer_svm = dx.Explainer(svm, X_train, y_train, predict_function=model_predicting_method, \
                             label="SVM_house_price")
explainer_xgboost = dx.Explainer(xgboost, X_train, y_train, predict_function=model_predicting_method, \
                                 label="XGBoost_house_price")

Preparation of a new explainer is initiated

  -> data              : 14480 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 14480 values
  -> model_class       : sklearn.svm._classes.SVR (default)
  -> label             : SVM_house_price
  -> predict function  : <function <lambda> at 0x7f4f00a9db90> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


/home/edsuch21/anaconda3/envs/myDalex/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  "X does not have valid feature names, but"


  -> predicted values  : min = -1.63e+03, mean = 5.06e+05, max = 3.1e+06
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -8.19e+05, mean = 3.19e+04, max = 4.72e+06
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 14480 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 14480 values
  -> model_class       : sklearn.ensemble._gb.GradientBoostingRegressor (default)
  -> label             : XGBoost_house_price
  -> predict function  : <function <lambda> at 0x7f4f00a9db90> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 1.91e+05, mean = 5.37e+05, max = 7.57e+06
  -> model type        : regression will be used (default)
  -> residual function : differen

/home/edsuch21/anaconda3/envs/myDalex/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [46]:
N = 100
nb_of_features = len(X_test.columns)

contributions_xgboost = np.zeros((N, nb_of_features))
contributions_svm = np.zeros((N, nb_of_features))

for i in range(0, N):
    for explainer, contributions in zip([explainer_xgboost, explainer_svm], \
                                        [contributions_xgboost, contributions_svm]):
        explain = explainer.predict_parts(X_test.iloc[0:1], type="unbiased_kernel_shap",
                                          n_samples = 500, label=f"sample id: 0")
        contr = explain.result['contribution'].to_numpy()
        contributions[i, :] = contr

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SV

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SV

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SV

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SV

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
X does not have valid feature names, but SVR was fitted with feature names
X does not have valid feature names, but SV

In [47]:
avg_xgboost = np.mean(contributions_xgboost, axis = 0)
std_xgboost = np.std(contributions_xgboost, axis = 0)
avg_svm = np.mean(contributions_svm, axis = 0)
std_svm = np.std(contributions_svm, axis = 0)

In [48]:
print(avg_xgboost)
print(std_xgboost)

[  3239.98067139    141.8940112  -60258.60282015  -5082.10055362
  -2557.82955755 -78300.42585429 -11201.43199118  10840.56257249
 -46056.81135389]
[19232.06534342 16103.88628907 27377.2221478  17467.86659807
 19527.67361082 33465.18843566 20897.70113738 19446.25971741
 20525.98880127]


In [49]:
print(avg_svm)
print(std_svm)

[  1705.62545722   1716.81433069 -85712.58527147   2397.78460234
   -166.88558784  -2056.8548628  -59097.90533122   5872.18060911
 -28966.67024678]
[12045.86253757 11373.00102613 29906.21897534 12212.7995395
 13778.13301255 13101.89118456 14656.81515204 12839.93870945
 16989.35562875]


In [66]:
exact_explainer_svm = shap.explainers.Exact(svm.predict, X_train)
exact_explainer_xgboost = shap.explainers.Exact(xgboost.predict, X_train)

exact_shap_result = []
for explainer in [exact_explainer_svm, exact_explainer_xgboost]:

    shap_values = explainer(X_test.iloc[0:1])

    exact_shap_result.append(shap_values.values)

Exact explainer: 2it [00:13, 13.25s/it]                                         


In [71]:
exact_shap_result[0]

array([[   393.76639509,     84.59555039, -79509.80981813,
           521.1875511 ,   -594.33166615,  -2367.61943529,
        -59133.5675569 ,   6778.20102502, -26083.84033299]])

In [72]:
np.sum(exact_shap_result[0])

-159911.4182878504

In [68]:
svm.predict(X_test.iloc[0:1])

array([336183.52288506])

In [54]:
np.sum(abs(exact_shap_result[0] - avg_svm)) / np.sum(exact_shap_result[0])

-0.09746754622057059

In [53]:
abs((exact_shap_result[1] - avg_xgboost) / exact_shap_result[1])

array([[0.6674807 , 1.09839402, 0.10429165, 0.46353489, 1.30364855,
        0.0866466 , 0.21073062, 0.25530479, 0.01341319]])